# Librerías y configuraciones


In [5]:
import pandas as pd
import numpy as np


In [6]:
pd.set_option('display.max_colwidth', 100)

---

# Carga de datos

In [7]:
df_genres = pd.read_csv('data/df_genres.csv')
df_item_genre = pd.read_csv('data/df_item_genre.csv')
# df_items = pd.read_csv('data/df_items.csv')
df_reviews = pd.read_csv('data/df_reviews.csv')
df_users_items = pd.read_csv('data/df_users_items.csv')
df_users = pd.read_csv('data/df_users.csv')
df_user_id_item_id_price = pd.read_csv("data/df_user_id_item_id_price.csv")
df_developer = pd.read_csv('data/df_developer.csv')

----

In [8]:
# 1. def userdata( User_id : str ): 
#     Debe devolver 
#     * cantidad de dinero gastado por el usuario,  
#     * el porcentaje de recomendación en base a reviews.recommend
#     * cantidad de items.


In [9]:
def userdata(user_id: str):
    # Si el usuario no existe devuelve: {'dinero_gastado': 0.0, 'porcentaje_de_recomendaciones': nan, 'items_recomendados': 0, 'items_comprados': 0}
    
    user_id = user_id.strip()

    if any(part in user_id for part in df_user_id_item_id_price['user_id']):

        # Dinero gastado
        df_filtrado_dinero = df_user_id_item_id_price[df_user_id_item_id_price.user_id == user_id]
        dinero_gastado = round(df_filtrado_dinero.price.sum(),2)

        # Porcentaje de recomendaciones
        df_filtrado_recomendaciones = df_reviews[df_reviews.user_id == user_id]
        
        if df_filtrado_recomendaciones.shape[0] == 0:
            porcentaje_de_recomendacion = 0
        else:
            porcentaje_de_recomendacion = round(df_filtrado_recomendaciones.recommend.mean(),3)*100
        
        # items comprados
        items_comprados = int(df_filtrado_dinero.shape[0])
        
        # items recomendados
        items_recomendados = int(df_filtrado_recomendaciones.shape[0])

        out = {"dinero_gastado":dinero_gastado, "porcentaje_de_recomendaciones": porcentaje_de_recomendacion, 'items_recomendados': items_recomendados,'items_comprados':items_comprados}

    else:
         out = (F'El usuario {user_id}, no éxiste. Intente nuevamente. EJ: "76561197970982479"')

    return out

In [10]:
userdata('76561197970982479')

{'dinero_gastado': 2772.39,
 'porcentaje_de_recomendaciones': 100.0,
 'items_recomendados': 3,
 'items_comprados': 169}

In [31]:
df_reviews[df_reviews.user_id == '76561197970982479']

,user_id,item_id,helpful,recommend,posted_date,year,sentiment_analysis
0,76561197970982479,1250,No ratings yet,True,2011-11-05,2011.0,2
1,76561197970982479,22200,No ratings yet,True,2011-07-15,2011.0,2
2,76561197970982479,43110,No ratings yet,True,2011-04-21,2011.0,2


In [11]:
df_user_id_item_id_price[df_user_id_item_id_price.user_id == '0212030']

,user_id,item_id,price
2345493,0212030,4560,19.99
2345494,0212030,20540,19.99
2345495,0212030,10500,19.99
2345496,0212030,730,14.99
2345497,0212030,8930,29.99


In [12]:
df_reviews[df_reviews.user_id == '0212030']

,user_id,item_id,helpful,recommend,posted_date,year,sentiment_analysis


----

2. def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): 
    * Cantidad de usuarios que realizaron reviews entre las fechas dadas y, 
    * el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [13]:
####### --------------->>>  Función 2

def countreviews (fecha_inicio:(str), fecha_fin):

    df_filtrado = df_reviews[(df_reviews['posted_date'] >= fecha_inicio) & (df_reviews['posted_date'] <= fecha_fin)]
    cant_usuarios = df_filtrado.shape[0]
    porcentaje_de_recomendacion = round(df_filtrado.recommend.mean()*100,3) # andaba cuando era round(df_filtrado.recommend.mean(),3)*100
    return {"cantidad":cant_usuarios, "porcentaje":porcentaje_de_recomendacion}
    # return df_filtrado

In [14]:
# pruebas
countreviews('2011-11-05' , '2011-11-20')

{'cantidad': 14, 'porcentaje': 92.857}

In [15]:
# el df df_reviews tiene valores nulos en la columna posted_date. Revisar como influyen en la funcion, si los toma en cuenta, si los ignora, etc.
#  se podria agregar un verificador de fechas validas.

---

3. def genre( género : str ): 
    * Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [16]:
####### --------------->>>  Función 3

def genre (genero):
    genero_minusculas = genero.lower().strip()
    if genero_minusculas in list(df_genres.name):
        puesto = int(df_genres[df_genres.name == genero_minusculas].reset_index().at[0,'ranking'])
        salida = {"genero": genero_minusculas, "ranking":puesto}
    else:
        puesto = f'El genero {genero_minusculas} no se encuentra. Intente nuevamente. Ej: Action'
        salida = {puesto}
    return salida

In [17]:
# Pruebas
print(genre ('strategy'))
print(genre ('action '))
print(genre ('    ACTION'))
print(genre ('strateggggy'))

{'genero': 'strategy', 'ranking': 6}
{'genero': 'action', 'ranking': 1}
{'genero': 'action', 'ranking': 1}
{'El genero strateggggy no se encuentra. Intente nuevamente. Ej: Action'}


----


4.
def userforgenre( género : str ): 
        games.genres
        games.id (item_id)
    Top 5 de usuarios
        head(5)
    con más horas de juego en el género dado, 
        item_df.items.playtime_forever
    con su URL (del user) y 
        items_df.user_url
    user_id.
        items_df.user_id

        *tabla genre
        *tabla genre_item
        *tabla items:        
        *tabla user_item:
        *tabla user:

In [18]:
df_item_genre.head(2)

,item_id,genres
0,761140,action
1,761140,casual


In [19]:
df_users_items.head(2)

,user_id,item_id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,30,7


In [20]:
df_users.head(2)

,Unnamed: 0,user_id,items_count,user_url
0,0,76561197970982479,277,http://steamcommunity.com/profiles/76561197970982479
1,1,js41637,888,http://steamcommunity.com/id/js41637


In [21]:
# -->>>> funcion 4

def userforgenre( genero : str ): 
    genero_min = genero.lower().strip()

    if genero_min in list(df_item_genre.genres):

        lista_de_items_del_genero = list(df_item_genre[df_item_genre.genres == genero_min].item_id)
        df = df_users_items[df_users_items.item_id.isin(lista_de_items_del_genero)]

        top5 = df.groupby('user_id')['playtime_forever'].sum().reset_index()
        top5 = top5.sort_values('playtime_forever',ascending=False).head() # aca ta el user id y las hs de juego
        top5 = top5.merge(df_users[['user_id','user_url']], on='user_id', how='left')
        users_top5 = list(top5.user_id)
        user_url_top5 = list(top5.user_url)

        out = {"users_top5":users_top5}#, "user_url_top5":user_url_top5}
        return out

    else:
        out = {f'el género {genero} no éxiste. Intente nuevamente. EJ: "Action"'}
        return out

In [22]:
userforgenre('action')

{'users_top5': ['Sp3ctre',
  'shinomegami',
  'REBAS_AS_F-T',
  'Terminally-Chill',
  'DownSyndromeKid']}

-----

5.
def developer( desarrollador : str ): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de salida:
            Ejemplo de salida:
            Activision	
            Año	Contenido Free
            2023	27%
            2022	25%
            xxxx	xx%

In [23]:
df_developer.head(20)

,developer,price,year
0,kotoshiro,4.99,2018
1,secret level srl,0.00,2018
2,poolians.com,0.00,2017
3,彼岸领域,0.99,2017
4,trickjump games ltd,3.99,2018
5,poppermost productions,9.99,2018
6,poppermost productions,18.99,2018
7,poppermost productions,29.99,2018
8,stegalosaurus game development,10.99,2018
9,copperpick studio,3.99,2018


In [24]:
def developer(desarrollador : str ):
    '''Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de salida:'''

    desarrollador_minusculas = desarrollador.lower().strip()

    
    if any(part in desarrollador_minusculas for part in df_developer['developer']):
        
        df_filtrado = df_developer[df_developer.developer == desarrollador_minusculas]
        df_xanio = df_filtrado.groupby(['year']).size().reset_index(name=('cantidad_total'))
        df_filtrado_free = df_filtrado[df_filtrado.price == 0].groupby(['year']).size().reset_index(name=('cantidad_free'))
        df = df_xanio.merge(df_filtrado_free,how='left')
        df = df.fillna(0)

        df['porcentaje'] = (df.cantidad_free / df.cantidad_total * 100).round().astype(int)
        df['cantidad_free'] = df['cantidad_free'].astype('int64')

        anios = df_xanio['year'].tolist()
        cantidad_de_items_free = df['cantidad_free'].tolist()
        porcentaje_free_por_anio = df['porcentaje'].tolist()

        out = {
            "anios": anios,
            "cantidad_de_items_free": cantidad_de_items_free,
            "porcentaje_free_por_anio": porcentaje_free_por_anio
            }
        
    else:
         out = {F'El desarrollador {desarrollador}, no éxiste. Intente nuevamente. EJ: "stegalosaurus game development"'}

    return out

In [25]:
developer('stegalosaurus game development')

{'anios': [2016, 2017, 2018],
 'cantidad_total_de_items': [7, 14, 1],
 'porcentaje_free_por_anio': [29, 0, 0]}

In [33]:
df_filtrado = df_developer[df_developer.developer == 'stegalosaurus game development']
df_filtrado


,developer,price,year
8,stegalosaurus game development,10.99,2018
4867,stegalosaurus game development,4.99,2016
4868,stegalosaurus game development,1.99,2016
5990,stegalosaurus game development,1.99,2016
6870,stegalosaurus game development,1.99,2016
8657,stegalosaurus game development,10.99,2017
8785,stegalosaurus game development,2.99,2017
8788,stegalosaurus game development,2.99,2017
9746,stegalosaurus game development,1.99,2017
9809,stegalosaurus game development,0.99,2017


In [34]:
df_xanio = df_filtrado.groupby(['year']).size().reset_index(name=('cantidad_total'))
df_filtrado_free = df_filtrado[df_filtrado.price == 0].groupby(['year']).size().reset_index(name=('cantidad_free'))
df = df_xanio.merge(df_filtrado_free,how='left')
df = df.fillna(0)
df

,year,cantidad_total,cantidad_free
0,2016,7,2.0
1,2017,14,0.0
2,2018,1,0.0


--------------

def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

                    Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [26]:
def sentiment_analysis( anio : int ):

    lista_de_anios = list(df_reviews.year.unique())
    lista_de_anios = [int(x) for x in lista_de_anios if not np.isnan(x)]
    lista_de_anios = sorted(lista_de_anios)
    if anio in lista_de_anios:

        df_filtrado = df_reviews[df_reviews.year == anio].groupby(['sentiment_analysis']).size().reset_index(name=('cantidad'))

        negative = int(df_filtrado.iloc[0,1])
        neutral = int(df_filtrado.iloc[1,1])
        positive = int(df_filtrado.iloc[2,1])

        return {"Negative":negative,"Neutral":neutral,"Positive":positive}
    else:
        return {f'El año ingresado no se encuentra. Puede ingresar una de las siguientes opciones:{lista_de_anios}'}

In [27]:
sentiment_analysis(2011)

{'Negative': 85, 'Neutral': 66, 'Positive': 298}

In [28]:
df_reviews.year.unique()

array([2011., 2014., 2013.,   nan, 2015., 2012., 2010.])